# PART 5 - Objective 1 - Data Cleaning


In [1]:
import pandas as pd
import numpy as np
import os
import re

from datetime import datetime
import math
import time

import scipy.stats as stats
from sklearn.neighbors import KNeighborsRegressor
import glob

import nltk
from string import punctuation
from nltk.corpus import stopwords  
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
  
# Create WordNetLemmatizer object 
word_lemm_obj = WordNetLemmatizer() 
stop_words = set(stopwords.words('english'))  

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

[nltk_data] Downloading package punkt to /Users/aparajita/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# define missing values function to check datasets for how many missing values there are
def missing_values(df):
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'total # of rows': len(df),
                                      'total # of NaN': df.isnull().sum(),
                                      'percent missing': percent_missing})
    missing_value_df.sort_values('percent missing', inplace=True, ascending=False)
    return missing_value_df

In [ ]:
# ###### Option 2: skip running the above code and read in saved datasets from the above checkpoints
# target_dataframe_site = pd.read_csv('Saved Datasets/target_dataframe_site.csv')
# phorizon_data = pd.read_csv('Saved Datasets/site_phorizon_pivoted_df_agg.csv')
# sitepm_geomorph_ncss_climate_satellite_data = pd.read_csv('Saved Datasets/ssp_sitepm_ncss_geomorph_agg_prism_satellite.csv')
# print(phorizon_data.shape , sitepm_geomorph_ncss_climate_satellite_data.shape)

# #create a mapping for pedon (peiid, peiidref,siteiid,siteiidref,siteobsiid)
# ssp_final = pd.read_csv('Saved Datasets/ssp_final.csv')
# site_var_list = ['siteobsiid','peiidref','peiid','siteiid','siteiidref']
# site_map_ids = ssp_final[site_var_list]

In [ ]:
sitepm_geomorph_ncss_climate_satellite_data = pd.read_csv('Saved Datasets/ssp_sitepm_ncss_geomorph_agg_prism_satellite.csv')
phorizon_data = pd.read_csv('Saved Datasets/site_phorizon_pivoted_df_agg.csv')
#join feature data together

# <<< before
# lab_soil_data = sitepm_geomorph_ncss_climate_satellite_data.drop_duplicates(subset=['siteiid'])
# feature_data = pd.merge(lab_soil_data, phorizon_data, how='left', on='siteiid')

# >>> after
sitepm_geomorph_ncss_climate_satellite_data.drop_duplicates(subset=['siteiid'], inplace = True)
feature_data = pd.merge(sitepm_geomorph_ncss_climate_satellite_data, phorizon_data, how='left', on='siteiid')
del sitepm_geomorph_ncss_climate_satellite_data, phorizon_data

feature_data.head()

In [ ]:
## read target data (MLRA , ecoclass information)
target_dataframe_site = pd.read_csv('Saved Datasets/target_dataframe_site.csv')
target_dataframe_site.shape

In [ ]:
# change feature_data siteiid to int in order to merge
feature_data['siteiid'] = feature_data['siteiid'].astype(int)

### join features and target and create the final model data
modeling_data = pd.merge(feature_data,target_dataframe_site,
                         how='left', on='siteiid')
modeling_data.shape

In [ ]:
### modeling data description
# modeling_describe_df = modeling_data.describe(include='all')
# des = modeling_describe_df.T
# des['missing_percent'] = modeling_data.isna().mean()

### Drop Variable

In [ ]:
var_to_drop_df = pd.read_csv('Input Files/var_to_drop.csv')

In [ ]:
list_to_drop = var_to_drop_df['var_name'].tolist()
list_to_drop

In [ ]:
modeling_data.columns

In [ ]:
len(list_to_drop)

In [ ]:
print(modeling_data.shape)
modeling_data = modeling_data.drop(columns=list_to_drop)
print(modeling_data.shape)

### count instead of dummy

In [ ]:
variable_with_pmorigin = modeling_data.columns[modeling_data.columns.str.contains('pmorigin')]
variable_with_pmmodifier = modeling_data.columns[modeling_data.columns.str.contains('pmmodifier')]
variable_with_pmkind = modeling_data.columns[modeling_data.columns.str.contains('pmkind')]

In [ ]:
class count_keywords_cls():

    def __init__(self,dummy_var,var_name):
        self.dummy_vars = dummy_var
        self.var_name = var_name
        
    def subset_dummy_data(self,modeling_data):
        """
        input : modeling data
        output: subset of modeling with column names renamed
        
        This function subset the model data for variables we sent.
        Since these are the one-hot encoded variables their naming convention is varname_category e.g. pmkind_ash
        we only keep the category name in column
        """
        self.dummy_vars = list(set(self.dummy_vars)-{self.var_name+'_nan', self.var_name+'_OTHER'})
        print('model df = ',modeling_data.shape)
        dummy_df = modeling_data[self.dummy_vars + ['siteiid']].copy()
        
#       Note: due to nature of our data set it is possible for each siteid (row) to have multiple categories
#       so we add a suffix of ' '  for when we do a .dot product, it could be considered as a delimiter so
#       we can distingush the cateories

        dummy_df = dummy_df.add_suffix(' ')
        dummy_df.columns = dummy_df.columns.str.replace(self.var_name+'_','')
        print('dummy df shape after subset = ', dummy_df.shape)
        self.dummy_df = dummy_df
        
    def reconstruct_var(self):
        """
        takes the subseted function and reconstruct the categries from one-hot encoding by dot product of 
        columns with value 1 and column name.

        
        """

        cols_without_siteid = list(set(self.dummy_df.columns)-{'siteiid '})
        ### To perevent memory compromise : subset to prevent kernel restart
        self.dummy_df.loc[0:100000,self.var_name] = self.dummy_df.loc[0:100000,cols_without_siteid].dot(self.dummy_df[cols_without_siteid].columns)
        print('dummy df shape after 1 dot = ', self.dummy_df.shape)
        self.dummy_df.loc[100000:200000,self.var_name] = self.dummy_df.loc[100000:200000,cols_without_siteid].dot(self.dummy_df[cols_without_siteid].columns)
        print('dummy df shape after 2 dot = ', self.dummy_df.shape)
        self.dummy_df.loc[200000:300000,self.var_name] = self.dummy_df.loc[200000:300000,cols_without_siteid].dot(self.dummy_df[cols_without_siteid].columns)
        print('dummy df shape after 3 dot = ', self.dummy_df.shape)
        self.dummy_df.loc[300000:400000,self.var_name] = self.dummy_df.loc[300000:400000,cols_without_siteid].dot(self.dummy_df[cols_without_siteid].columns)
        print('dummy df shape after 4 dot = ', self.dummy_df.shape)
        self.dummy_df.loc[400000:,self.var_name] = self.dummy_df.loc[400000:,cols_without_siteid].dot(self.dummy_df[cols_without_siteid].columns)
        print('dummy df shape after 5 dot = ', self.dummy_df.shape)
        
    def cleaning_tokenizing(self):
        """
        This function lemmuniza and lower case the categories
        """
    
        self.dummy_df[self.var_name+'_lower'] = self.dummy_df[self.var_name].str.lower().astype(str)
        self.dummy_df[self.var_name+'_lemm_lower']= self.dummy_df[self.var_name+'_lower'].apply(lambda x:word_lemm_obj.lemmatize(x))
        dummy_df_final= self.dummy_df[['siteiid ',self.var_name+'_lemm_lower']]
        return(dummy_df_final)
    
    def count_keywords_func(self,dummy_df_final):
        """
        This function tokenize the 
        """
       
        
        ### data cleaning, replace - with ' '
        dummy_df_final[self.var_name+'_lemm_lower'] = dummy_df_final[self.var_name+'_lemm_lower'].apply(lambda x:x.replace('-',' '))
        print('getting list of keywords')
        KEYWORDS_LST = np.unique(nltk.word_tokenize(
            ' '.join(dummy_df_final[self.var_name+'_lemm_lower'].tolist())))
        
        filterd_keywords = [keyword for keyword in KEYWORDS_LST if 
                            ((keyword not in punctuation) & 
                             (not keyword.isdigit()) & 
                             (keyword not in(stop_words)))
                           ]
        print(' number of keywords is : ',len(filterd_keywords))
        print(' var name = ',self.var_name)
        for kw in filterd_keywords:
            print ('counting keywords for : ', kw)
            dummy_df_final[self.var_name+'_'+kw] = dummy_df_final[self.var_name+'_lemm_lower'].str.count(kw)
        dummy_df_final.drop(columns= [self.var_name+'_lemm_lower'],inplace=True)
        dummy_df_final.columns = dummy_df_final.columns.str.strip()
        
        return dummy_df_final
    
    def replace_in_model_df(self,count_var_df,modeling_data):
        modeling_data_in = modeling_data.copy()
        modeling_data_in.drop(columns= self.dummy_vars,inplace=True)
        new_model_data = pd.merge(modeling_data_in,count_var_df,on='siteiid',how='left')
        return (new_model_data)

In [ ]:
count_keywords_pmmodifier = count_keywords_cls(variable_with_pmmodifier,'pmmodifier')
count_keywords_pmmodifier.subset_dummy_data(modeling_data)
count_keywords_pmmodifier.reconstruct_var()
tokenized_df_pmmodifier = count_keywords_pmmodifier.cleaning_tokenizing()
count_df_pmmodifier = count_keywords_pmmodifier.count_keywords_func(tokenized_df_pmmodifier)
print('dropping duplicated siteiid')
count_df_pmmodifier.drop_duplicates(subset=['siteiid'],inplace=True)
print('merge')
new_model_data_2 = count_keywords_pmmodifier.replace_in_model_df(count_df_pmmodifier,modeling_data)
new_model_data_2.shape

# >>> after
del count_keywords_pmmodifier, tokenized_df_pmmodifier, count_df_pmmodifier

In [ ]:
count_keywords_pmkind = count_keywords_cls(variable_with_pmkind,'pmkind')
count_keywords_pmkind.subset_dummy_data(new_model_data_2)
count_keywords_pmkind.reconstruct_var()
tokenized_df_pmkind = count_keywords_pmkind.cleaning_tokenizing()
count_df_pmkind = count_keywords_pmkind.count_keywords_func(tokenized_df_pmkind)
count_df_pmkind.drop_duplicates(subset=['siteiid'],inplace=True)
new_model_data_3 = count_keywords_pmkind.replace_in_model_df(count_df_pmkind,new_model_data_2)
new_model_data_3.shape

# >>> after
del count_keywords_pmkind, tokenized_df_pmkind, count_df_pmkind

In [ ]:
count_keywords_pmorigin = count_keywords_cls(variable_with_pmorigin,'pmorigin')
count_keywords_pmorigin.subset_dummy_data(new_model_data_3)
count_keywords_pmorigin.reconstruct_var()
tokenized_df_pmorigin = count_keywords_pmorigin.cleaning_tokenizing()
count_df_pmorigin = count_keywords_pmorigin.count_keywords_func(tokenized_df_pmorigin)
count_df_pmorigin.drop_duplicates(subset=['siteiid'],inplace=True)

# <<< before
# new_model_data_4 = count_keywords_pmorigin.replace_in_model_df(count_df_pmorigin,new_model_data_3)
# new_model_data_4.shape

# >>> after
modeling_data = count_keywords_pmorigin.replace_in_model_df(count_df_pmorigin,new_model_data_3)
# modeling_data.shape
del count_keywords_pmorigin, tokenized_df_pmorigin, count_df_pmorigin

### Discretization

In [ ]:
def discretize_claytotest(var_name,modeling_data):
    
    """
    Discretize to create 4 new flags: clay<=10 (=1 if >=0 and <=10, else 0); 
    clay10to20 (=1 if >10 and <=20, else 0); clay20to30 (=1 if >20 and <=30, else 0); 
    clay>30 (=1 if >30, else 0); missing values =0 for all new flags 
    
    """
    
    modeling_data[var_name+'<10'] = modeling_data[var_name].apply(lambda x: 
                                                            1 if ((x <= 10)&(x >= 0)) else 0)
    modeling_data[var_name+'_10to20'] = modeling_data[var_name].apply(lambda x: 
                                                            1 if ((x <= 20)&(x > 10)) else 0)
    modeling_data[var_name+'_20to30'] = modeling_data[var_name].apply(lambda x: 
                                                            1 if ((x <= 30)&(x > 20)) else 0)
    modeling_data[var_name+'>30'] = modeling_data[var_name].apply(lambda x: 
                                                            1 if (x > 30) else 0)
    modeling_data.drop(columns=[var_name],inplace=True)

In [ ]:
claytotes_vars = modeling_data.columns[modeling_data.columns.str.contains('claytotest_')]
for claytotest_var in claytotes_vars:
    print(claytotest_var)
    discretize_claytotest(claytotest_var, modeling_data)
    
# >>> after
del claytotes_vars

In [ ]:
def discritize_fragvoltot(var_name,modeling_data):
    
    """
    Discretize to create 2 new flags: frag<=10 (=1 if >=0 and <=10, else 0); 
    frag>10 (=1 if >10; else 0); 
    missing values =0 for all new flags
    
    """
    modeling_data[var_name+'<10'] = modeling_data[var_name].apply(lambda x: 
                                                            1 if ((x <= 10)&(x >= 0)) else 0)
    modeling_data[var_name+'>10'] = modeling_data[var_name].apply(lambda x: 
                                                            1 if (x > 10) else 0)
    modeling_data.drop(columns=[var_name],inplace=True)

In [ ]:
fragvoltot_vars = modeling_data.columns[modeling_data.columns.str.contains('fragvoltot')]
for fragvoltot_var in fragvoltot_vars:
    print(fragvoltot_var)
    discritize_fragvoltot(fragvoltot_var,modeling_data)
    
# >>> after
del fragvoltot_vars

In [ ]:
def discritize_phfield(var_name,modeling_data):
    
    """
    Discretize using scale to right to create 11 new flags; 
    missing values are 0 for all flags
    
    """
    
    modeling_data[var_name+'_strong_acidic'] = modeling_data[var_name].apply(lambda x: 
                                         1 if (x <= 5.5) else 0)
    modeling_data[var_name+'_moderate_acidic'] = modeling_data[var_name].apply(lambda x: 
                                    1 if ((x <= 6)&(x >= 5.6)) else 0)
    modeling_data[var_name+'_slight_acidic'] = modeling_data[var_name].apply(lambda x: 
                                    1 if ((x <= 6.5)&(x >= 6.1)) else 0)
    modeling_data[var_name+'_neutral'] = modeling_data[var_name].apply(lambda x: 
                                    1 if ((x <= 7.3)&(x >= 6.6)) else 0)
    modeling_data[var_name+'_slight_alkaline'] = modeling_data[var_name].apply(lambda x: 
                                    1 if ((x <= 7.8)&(x >= 7.4)) else 0)
    modeling_data[var_name+'_moderate_alkaline'] = modeling_data[var_name].apply(lambda x: 
                                    1 if ((x <= 8.4)&(x >= 7.9)) else 0)
    modeling_data[var_name+'_strong_alkaline'] = modeling_data[var_name].apply(lambda x: 
                                    1 if (x >= 8.5) else 0)    
    modeling_data.drop(columns=[var_name],inplace=True)

In [ ]:
phfild_vars = modeling_data.columns[modeling_data.columns.str.contains('phfield')]
for phfild_var in phfild_vars:
    print(phfild_var)
    discritize_phfield(phfild_var,modeling_data)
    
# >>> after
del phfild_vars

In [ ]:
# modeling_data['phfield_0_moderate_acidic'].unique()

### Replace lat, long , elev with new data

In [ ]:
lat_long_elev_data = pd.read_csv('Input Files/siteiid_lat_long_elev.csv')

In [ ]:
geographic_loc_info = ['latstddeci','latstddecimaldegrees','longstddec','longstddecimaldegrees','elev']
modeling_data.drop(columns = geographic_loc_info,inplace=True)  

In [ ]:
geo_df = lat_long_elev_data[['siteiid','latstddeci','longstddec','mn75_grd']].drop_duplicates()
geo_df.rename(columns={'mn75_grd':'elev'},inplace=True)
geo_df.shape, geo_df['siteiid'].nunique()

In [ ]:
modeling_data['siteiid'] = modeling_data['siteiid'].astype(int) #make data type int to match geo_df siteiid then change back to string later

# <<< before
# modeling_data_replace_geo = pd.merge(modeling_data,
#                                      geo_df,
#                                      on='siteiid',how='left')

# >>> after
modeling_data = pd.merge(modeling_data,
                         geo_df,
                         on='siteiid',how='left')

### Fill in Null values

#### Fill with zero

In [ ]:
var_to_fill_zero_df = pd.read_csv('Input Files/var_to_fill_zero_data.csv')
missing_var_list = var_to_fill_zero_df['var_name'].tolist()

In [ ]:
modeling_data[missing_var_list] = modeling_data[
    missing_var_list].apply(lambda x: x.fillna(0))

#### Fill with median

In [ ]:
var_to_fill_with_median = ['hzdept_0','hzdept_10','hzdept_20','hzdept_30','hzdept_40','hzdept_50','hzdept_60',
 'hzdept_70','hzdept_80','hzdept_90','hzdept_100','hzdept_110','hzdept_120',
'hzdepb_0','hzdepb_10','hzdepb_20','hzdepb_30','hzdepb_40','hzdepb_50','hzdepb_60',
'hzdepb_70','hzdepb_80','hzdepb_90','hzdepb_100','hzdepb_110','hzdepb_120']

In [ ]:
modeling_data[var_to_fill_with_median] = modeling_data[
    var_to_fill_with_median].apply(lambda x: x.fillna(x.median()))

### Create Flag

In [ ]:
#Feature engineer flag noncarbclay (=0 if Nan; 1 otherwise)

flag_var = ['noncarbclaywtavg','claytotwtavg',
            'cec7clayratiowtavg','pmorder','psctopdepth']


In [ ]:
for var in flag_var:
    modeling_data.loc[modeling_data[var].notna(),var] = 1
    modeling_data.loc[modeling_data[var].isna(),var] = 0

### Group together

In [ ]:
# Let's group these together into a new flag 
# statextsflag_0to30 (=1 if _0,10,20, or 30 are 1, else 0)

stratextsflag_vars = ['stratextsflag_0','stratextsflag_10',
                      'stratextsflag_20','stratextsflag_30']

modeling_data[stratextsflag_vars] = modeling_data[
    stratextsflag_vars].apply(lambda x: x.fillna(0))

modeling_data['statextsflag_0to30'] = modeling_data[stratextsflag_vars].any(axis=1).map(
    {True:1,False:0})

modeling_data.drop(columns=stratextsflag_vars,inplace=True)

In [ ]:
# Group _0 to _70 into a new flag horcolorvflag_0to70 
#                       (=1 if any of _0 to _70 =1; else 0); 
# this will convert missing to 0 as well

horcolorvflag_vars = ['horcolorvflag_0', 'horcolorvflag_10', 'horcolorvflag_20', 
                      'horcolorvflag_30', 'horcolorvflag_40', 'horcolorvflag_50',
                      'horcolorvflag_60', 'horcolorvflag_70']

modeling_data[horcolorvflag_vars] = modeling_data[
    horcolorvflag_vars].apply(lambda x: x.fillna(0))

modeling_data['horcolorvflag_0to70'] = modeling_data[horcolorvflag_vars].any(axis=1).map(
    {True:1,False:0})

modeling_data.drop(columns=horcolorvflag_vars,inplace=True)




### OTHER CLEANING

In [ ]:
## drop count of Nan variables
nan_vars = modeling_data.columns[modeling_data.columns.str.endswith('_nan')]
# drop variables that have _OTHER and are less than 0.01 of column
other_vars = [
'obsmethod_0_OTHER',
'obsmethod_100_OTHER',
'obsmethod_70_OTHER',
'obsmethod_30_OTHER',
'obsmethod_80_OTHER',
'obsmethod_120_OTHER',
'obsmethod_110_OTHER',
'obsmethod_20_OTHER',
'obsmethod_90_OTHER',
'obsmethod_40_OTHER',
'obsmethod_50_OTHER'
]

modeling_data.drop(columns=nan_vars,inplace=True)
modeling_data.drop(columns=other_vars,inplace=True)

In [ ]:
## fill NAN values with zeros for the following features
fill_with_zero_feat = [
    'Feature_Type_Anthropogenic Feature', 'Feature_Type_Landform', 'Feature_Type_Landscape', 
    'Feature_Type_Microfeature', 'Feature_alluvial fan', 'Feature_coastal plain', 'Feature_drainageway', 
    'Feature_flood plain', 'Feature_foothills', 'Feature_ground moraine', 'Feature_hill', 'Feature_hills', 
    'Feature_hillslope', 'Feature_interfluve', 'Feature_intermontane basin', 'Feature_lake plain', 
    'Feature_mountain', 'Feature_mountain slope', 'Feature_mountains', 'Feature_other', 'Feature_outwash plain',
    'Feature_piedmont', 'Feature_plain', 'Feature_plains', 'Feature_plateau', 'Feature_ridge', 
    'Feature_river valley', 'Feature_stream terrace', 'Feature_terrace', 'Feature_till plain', 
    'Feature_upland', 'Feature_valley', 'obsmethod_0_Bucket Auger', 'obsmethod_50_Large Pit or Quarry', 
    'obsmethod_110_Push Tube', 'obsmethod_10_Shovel Slice', 'obsmethod_20_Push Tube', 'obsmethod_30_Small Pit', 
    'obsmethod_20_Small Pit', 'obsmethod_80_Bucket Auger', 'hzname_20_OTHER', 'hzname_70_OTHER',
    'obsmethod_60_Trench', 'hzname_10_OTHER', 'obsmethod_120_Bucket Auger', 'obsmethod_40_Small Pit',
    'obsmethod_0_Cut', 'desgnmaster_50_OTHER', 'obsmethod_10_Small Pit', 'obsmethod_20_Cut', 
    'obsmethod_10_Push Tube', 'obsmethod_70_Cut', 'obsmethod_50_Push Tube', 'obsmethod_30_Shovel Slice', 
    'obsmethod_90_Screw Auger', 'obsmethod_50_Small Pit', 'obsmethod_10_Large Pit or Quarry', 'obsmethod_50_Cut',
    'hzname_0_OTHER', 'obsmethod_120_Cut', 'obsmethod_110_Trench', 'obsmethod_90_Push Tube', 
    'obsmethod_20_Large Pit or Quarry', 'obsmethod_40_Shovel Slice', 'obsmethod_10_Trench', 'hzname_90_OTHER', 
    'obsmethod_120_Trench', 'obsmethod_100_Small Pit', 'obsmethod_0_Small Pit', 'obsmethod_0_Shovel Slice', 
    'obsmethod_120_Push Tube', 'obsmethod_30_Large Pit or Quarry', 'obsmethod_40_Cut', 
    'obsmethod_120_Large Pit or Quarry', 'obsmethod_30_Bucket Auger', 'obsmethod_60_Push Tube', 
    'obsmethod_20_Shovel Slice', 'obsmethod_30_Push Tube', 'obsmethod_70_Trench', 'obsmethod_0_Trench',
    'obsmethod_60_Cut', 'obsmethod_40_Large Pit or Quarry', 'hzname_40_OTHER', 'obsmethod_80_Trench',
    'hzname_60_OTHER', 'obsmethod_100_Trench', 'hzname_120_OTHER', 'obsmethod_100_Cut', 'obsmethod_80_Small Pit', 
    'desgnmaster_0_OTHER', 'obsmethod_110_Bucket Auger', 'obsmethod_60_Small Pit', 'obsmethod_90_Trench', 
    'obsmethod_90_Cut', 'desgnmaster_110_OTHER', 'desgnmaster_80_OTHER', 'obsmethod_90_Bucket Auger', 
    'desgnmaster_100_OTHER', 'obsmethod_30_Trench', 'obsmethod_80_Large Pit or Quarry', 
    'desgnmaster_10_OTHER', 'obsmethod_10_Bucket Auger', 'obsmethod_40_Trench',
    'obsmethod_110_Large Pit or Quarry', 'obsmethod_60_Bucket Auger', 'hzname_80_OTHER', 'hzname_50_OTHER', 
    'obsmethod_50_Bucket Auger', 'obsmethod_70_Large Pit or Quarry', 'obsmethod_70_Push Tube', 
    'desgnmaster_40_OTHER', 'obsmethod_70_Small Pit', 'obsmethod_110_Small Pit', 'obsmethod_10_Cut',
    'obsmethod_80_Cut', 'obsmethod_60_Large Pit or Quarry', 'desgnmaster_90_OTHER', 'obsmethod_110_Cut', 
    'obsmethod_20_Bucket Auger', 'desgnmaster_20_OTHER', 'obsmethod_100_Bucket Auger', 'obsmethod_80_Screw Auger',
    'obsmethod_20_Trench', 'obsmethod_100_Large Pit or Quarry', 'obsmethod_50_Shovel Slice',
    'desgnmaster_70_OTHER', 'obsmethod_10_OTHER', 'obsmethod_60_OTHER', 'obsmethod_80_Push Tube',
    'obsmethod_0_Large Pit or Quarry', 'desgnmaster_30_OTHER', 'obsmethod_40_Bucket Auger', 
    'obsmethod_90_Large Pit or Quarry', 'hzname_100_OTHER', 'obsmethod_50_Trench', 'hzname_110_OTHER', 
    'obsmethod_120_Small Pit', 'obsmethod_70_Bucket Auger', 'obsmethod_90_Small Pit', 'obsmethod_0_Push Tube',
    'obsmethod_100_Screw Auger', 'obsmethod_100_Push Tube', 'desgnmaster_60_OTHER', 'hzname_30_OTHER', 
    'obsmethod_30_Cut', 'obsmethod_40_Push Tube']


modeling_data[fill_with_zero_feat] = modeling_data[
    fill_with_zero_feat].apply(lambda x: x.fillna(0))

### Join with Veg

#### Create reference table for plant/trees
Need this table in order to join the vegplot and plant datasets

In [ ]:
# Vegetation data
windbreakrowdata = pd.read_csv('Input Files/windbreakrowdata.txt', delimiter="|")
plottreeinventory = pd.read_csv('Input Files/plottreeinventory.txt', delimiter="|")
plotplantinventory = pd.read_csv('Input Files/plotplantinventory.txt', delimiter="|")
plottreesiteindexsummary = pd.read_csv('Input Files/plottreesiteindexsummary.txt', delimiter="|")

In [ ]:
pv_1 = windbreakrowdata[['plantiidref', 'vegplotiidref']]
pv_2 = plottreesiteindexsummary[['plantiidref', 'vegplotiidref']]
pv_3 = plottreeinventory[['plantiidref', 'vegplotiidref']]
pv_4 = plotplantinventory[['plantiidref', 'vegplotiidref']]

frames = [pv_1, pv_2, pv_3, pv_4]
pv_table = pd.concat(frames)
pv_table.drop_duplicates(inplace=True)
del windbreakrowdata, plottreeinventory, plotplantinventory, plottreesiteindexsummary
del pv_1, pv_2, pv_3, pv_4

# drop duplicates
# pv_table_final = pv_table.drop_duplicates()

pv_table.head()

#### cleanse vegplot

In [ ]:
vegplot = pd.read_csv('Input Files/vegplot.txt', delimiter="|")

In [ ]:
# preview raw data
vegplot.head()

In [ ]:
# missing values
missing_values(vegplot)

In [ ]:
# keep variables and drop the rest
vegplot = vegplot[['vegplotiid',
                  'soilprofileindicator',
                  'alkalineaffected',
                  'understorydescindicator',
                  'mensurationdataindicator',
                  'siteobsiidref']]
vegplot.drop_duplicates(inplace=True)

# vegplot_final = vegplot_v2.drop_duplicates()

#### plant

In [ ]:
plant = pd.read_csv('Input Files/plant.txt', delimiter="|", encoding='latin-1')

In [ ]:
# preview raw plant dataset
plant.head()

In [ ]:
# missing values
missing_values(plant)

In [ ]:
# drop variables that are more than 70% missing and/or are not useful for analysis
plant = plant.drop(columns = ['obterm',
                             'plantdbiidref',
                             'grpiidref', 
                             'objwlupdated',
                             'objuseriidref',
                             'recwlupdated',
                             'recuseriidref',
                             'plantsubspecies',
                             'plantvariety'])

In [ ]:
plant.drop_duplicates(inplace=True)
plant.shape

#### pv + vegplot

In [ ]:
pv_vegplot = pd.merge(pv_table, 
                    vegplot, 
                    how='left', 
                    left_on=['vegplotiidref'], 
                    right_on=['vegplotiid'],
                    suffixes=('_pv','_vegplot'))

pv_vegplot.shape

#### +plant

In [ ]:
pv_vegplot_plant = pd.merge(pv_vegplot, 
                    plant, 
                    how='inner', 
                    left_on=['plantiidref'], 
                    right_on=['plantiid'],
                    suffixes=('_pv','_plant'))

pv_vegplot_plant.shape

In [ ]:
pv_vegplot_plant.drop(columns = ['plantiidref',
                                'vegplotiidref',
                                'vegplotiid',
                                'plantiid'], inplace=True)

In [ ]:
pv_vegplot_plant.drop_duplicates(inplace=True)
pv_vegplot_plant.shape

In [ ]:
pv_vegplot_plant.head()

In [ ]:
# fill missing plantsciname with plantnatvernm
pv_vegplot_plant['plantsciname'].fillna(pv_vegplot_plant['plantnatvernm'], inplace=True)

##### One hot encode

In [ ]:
# one hot encode - get variables
plantsciname = pv_vegplot_plant['plantsciname'].value_counts().to_frame()
features = plantsciname[plantsciname['plantsciname'] > 5000].reset_index()
features = features.drop(columns = 'plantsciname')
features = features.rename(columns={"index": "Features"})
features

In [ ]:
pv_vegplot_plant_v2 = pv_vegplot_plant.drop(columns = ['plantsym', 
                                                       'plantnatvernm',
                                                       'plantgenus',
                                                       'plantspecies'])

pv_vegplot_plant_v3 = pd.merge(pv_vegplot_plant_v2, 
                               features, 
                               how='left', 
                               left_on=['plantsciname'], 
                               right_on=['Features'])
pv_vegplot_plant_v3



In [ ]:
# fill in NaN in column Features with "Other"
values = {'Features': 'Other'}
pv_vegplot_plant_v3 = pv_vegplot_plant_v3.fillna(value=values)
pv_vegplot_plant_v3

In [ ]:
# One Hot Encoding
pv_vegplot_plant_v4 = pv_vegplot_plant_v3[['siteobsiidref']].join(pd.get_dummies(pv_vegplot_plant_v3['Features']).add_prefix('PlantName_')).groupby('siteobsiidref').max().reset_index()
pv_vegplot_plant_v4

In [ ]:
pv_vegplot_plant_v4['siteobsiidref'] = pv_vegplot_plant_v4['siteobsiidref'].astype('str')

##### join siteid

In [ ]:
#create a mapping for pedon (peiid, peiidref,siteiid,siteiidref,siteobsiid)
ssp_final = pd.read_csv('Saved Datasets/ssp_final.csv')

# <<< before
# site_var_list = ['siteobsiid','peiidref','peiid','siteiid','siteiidref']

# >>> after
site_var_list = ['siteobsiid','siteiid']

site_map_ids = ssp_final[site_var_list]
site_ids = site_map_ids

In [ ]:
# <<< before
# site_ids.drop(columns = ['peiidref', 'peiid', 'siteiidref'], inplace=True)
# site_ids.head()

In [ ]:
# change to integer in order to join
pv_vegplot_plant_v4['siteobsiidref'] = pv_vegplot_plant_v4['siteobsiidref'].astype(int)

plantname = pd.merge(pv_vegplot_plant_v4,
                           site_ids,
                           how='inner',
                           left_on=['siteobsiidref'], 
                           right_on=['siteobsiid'])
plantname.shape

##### collapse on siteiid

In [ ]:
plantname_final = plantname.groupby('siteiid').max().reset_index()

In [ ]:
plantname_final['siteiid'] = plantname_final['siteiid'].astype(int)

In [ ]:
plantname_final.shape

In [ ]:
plantname_final.head()

In [ ]:
plantname_final.drop(columns='siteobsiid',inplace=True )

In [ ]:
modeling_data_veg = modeling_data.merge(plantname_final, how='left', on = 'siteiid')

In [ ]:
modeling_data.shape,modeling_data_veg.shape

In [ ]:
fill_veg_vars = list(set(list(plantname_final))-{'siteiid'})
modeling_data_veg[fill_veg_vars] = modeling_data_veg[
    fill_veg_vars].apply(lambda x: x.fillna(0))

modeling_data = modeling_data_veg.copy()
del modeling_data_veg

### Drop Vars

In [ ]:
### drop texcl variables because the information is already repeating in texture varibles with less missing %
variable_with_texcl = modeling_data.columns[modeling_data.columns.str.startswith('texcl')]
modeling_data.drop(columns=variable_with_texcl,inplace=True)

In [ ]:
### drop these extra variables
drop_vars = [          
'rupresblkmst_80_Extremely firm','rupresblkmst_90_Extremely firm','rupresblkmst_100_Extremely firm',
'rupresblkmst_110_Extremely firm','rupresblkmst_120_Extremely firm',
'earthcov_1_Marshland',
'PlantName_Abies grandis',
'PlantName_Larix occidentalis',
'PlantName_Pseudotsuga menziesii var. glauca',
'PlantName_Tsuga heterophylla',
'pmorigin_andesite',
'pmorigin_gneiss',
'pmorigin_granitoid',
'pmorigin_metasedimentary',
'pmorigin_mudstone',
'pmorigin_tuff',
'siteobsiidref']
modeling_data.drop(columns=drop_vars,inplace=True)
modeling_data.shape

In [ ]:
#drop_less_frequent (less than 1%):
less_freq =   ['pmgroupnam_till',
    'earthcov_1_Other tree cover', 'earthcov_1_Savanna rangeland', 'Feature_Type_Anthropogenic Feature',
    'Feature_Type_Microfeature',
    'horcolorvflag_80', 'horcolorvflag_90', 'horcolorvflag_100', 'horcolorvflag_110', 'horcolorvflag_120',
    'obsmethod_50_Large Pit or Quarry',
    'hzname_60_2Bt2', 'hzname_40_E',
    'obsmethod_10_Shovel Slice', 'desgnmaster_60_O', 'hzname_0_Oa', 'rupresblkmst_50_Extremely firm', 
    'obsmethod_60_Trench', 'hzname_120_Bk', 'obsmethod_0_Cut', 'desgnmaster_90_O', 'obsmethod_20_Cut',
     'hzname_110_2C2', 'obsmethod_70_Cut', 'effclass_60_Very slightly effervescent', 'obsmethod_30_Shovel Slice', 
     'obsmethod_90_Screw Auger', 'obsmethod_10_Large Pit or Quarry', 'desgnmaster_30_AB', 
     'effclass_30_Very slightly effervescent', 'hzname_50_2Bt2', 'desgnmaster_60_E', 
     'effclass_50_Very slightly effervescent', 'obsmethod_50_Cut', 'obsmethod_120_Cut', 'obsmethod_110_Trench', 
     'obsmethod_20_Large Pit or Quarry', 'obsmethod_40_Shovel Slice', 'obsmethod_10_Trench', 'hzname_30_BC',
     'obsmethod_120_Trench', 'obsmethod_0_Shovel Slice', 'effclass_110_Very slightly effervescent', 
     'obsmethod_30_Large Pit or Quarry', 'desgnmaster_80_O', 'obsmethod_40_Cut', 'obsmethod_120_Large Pit or Quarry',
     'effclass_40_Very slightly effervescent', 'rupresblkmst_60_Extremely firm', 'desgnmaster_50_O', 'hzname_40_Bt3',
     'hzname_10_R', 'hzname_30_BA', 'hzname_70_Btk', 'obsmethod_20_Shovel Slice', 'obsmethod_70_Trench',
     'obsmethod_0_Trench', 'obsmethod_60_Cut', 'obsmethod_40_Large Pit or Quarry', 'hzname_40_Btk', 
     'desgnmaster_40_O', 'obsmethod_80_Trench', 'rupresblkmst_0_Very firm', 'desgnmaster_70_E', 'desgnmaster_20_BE', 
     'obsmethod_100_Trench', 'hzname_80_Bw', 'obsmethod_100_Cut', 'hzname_30_Cr', 
     'effclass_80_Very slightly effervescent', 'hzname_110_Cg', 'hzname_80_C3', 'hzname_30_AB', 
     'obsmethod_90_Trench', 'obsmethod_90_Cut', 'desgnmaster_10_R', 'obsmethod_30_Trench', 'hzname_50_A', 
     'boundtopo_0_Irregular', 'effclass_120_Very slightly effervescent', 'obsmethod_80_Large Pit or Quarry', 
     'effclass_90_Very slightly effervescent', 'obsmethod_40_Trench', 'hzname_0_C1',
     'obsmethod_110_Large Pit or Quarry', 'hzname_70_C3', 'bounddistinct_40_Diffuse', 
     'obsmethod_70_Large Pit or Quarry', 'desgnmaster_120_A', 'obsmethod_10_Cut', 'obsmethod_80_Cut', 
     'obsmethod_60_Large Pit or Quarry', 'obsmethod_110_Cut', 'effclass_100_Very slightly effervescent', 
     'obsmethod_80_Screw Auger', 'obsmethod_20_Trench', 'obsmethod_100_Large Pit or Quarry',
     'obsmethod_50_Shovel Slice', 'obsmethod_10_OTHER', 'desgnmaster_30_BA', 'obsmethod_60_OTHER', 
     'obsmethod_0_Large Pit or Quarry', 'hzname_50_Btk', 'bounddistinct_30_Diffuse', 'desgnmaster_110_A', 
     'hzname_120_Bt3', 'hzname_80_2Bt3', 'obsmethod_90_Large Pit or Quarry', 'obsmethod_50_Trench',
     'rupresblkmst_70_Extremely firm', 'bounddistinct_20_Diffuse', 'desgnmaster_70_O', 'hzname_50_A2', 
     'hzname_100_Bt', 'hzname_60_Btk', 'obsmethod_100_Screw Auger', 'hzname_120_Bk2', 
     'effclass_70_Very slightly effervescent', 'obsmethod_30_Cut', 'pmkind_glaciolacustrine',
     'pmorigin_cherty', 'pmorigin_dolomite', 'pmorigin_quartzite', 'pmorigin_schist', 'pmorigin_volcanic', 
     'phfield_0_strong_alkaline', 'PlantName_Acer saccharum', 'PlantName_Aristida', 
     'PlantName_Calamagrostis canadensis', 'PlantName_Calamagrostis rubescens', 'PlantName_Chrysothamnus', 
     'PlantName_Elymus elymoides', 'PlantName_Koeleria macrantha', 'PlantName_Lupinus', 
     'PlantName_Pascopyrum smithii', 'PlantName_Pinus contorta', 'PlantName_Pinus ponderosa', 
     'PlantName_Pinus strobus', 'PlantName_Poa', 'PlantName_Populus tremuloides', 
     'PlantName_Prosopis glandulosa var. torreyana', 'PlantName_Pseudoroegneria spicata', 
     'PlantName_Quercus alba', 'PlantName_Quercus rubra', 'PlantName_Sporobolus cryptandrus',
     'PlantName_Symphoricarpos albus', 
     'texture_60_VFSL', 'texture_0_GR-SIL', 'texture_90_LFS', 'texture_100_VFSL', 'texture_90_VFSL', 
     'texture_0_GRV-L', 'texture_120_LS', 'texture_30_GRV-L', 'texture_120_VFSL', 'texture_110_LFS',
     'texture_50_VFSL', 'texture_10_GRV-L', 'texture_40_VFSL', 'texture_100_LFS', 'texture_0_GR-SL',
     'texture_120_LFS', 'texture_20_GRV-L', 'texture_60_LFS', 'texture_70_LFS', 'texture_10_GR-SIL',
     'texture_80_LFS', 'texture_110_VFSL', 'texture_30_GR-L', 'texture_80_VFSL', 'texture_70_VFSL']

modeling_data.drop(columns=less_freq,inplace=True)
modeling_data.shape

### Drop Index variable 

In [ ]:
## first we impute the NaN values with kmeans (based on location because climate is location and elev deopendent)
vars_to_impute = ['ppt01', 'ppt02', 'ppt03', 'ppt04', 'ppt05', 'ppt06', 'ppt07', 'ppt08', 'ppt09', 'ppt10', 'ppt11', 'ppt12', 'pptannual', 

'tdmean01', 'tdmean02', 'tdmean03', 'tdmean04', 'tdmean05', 'tdmean06', 'tdmean07', 'tdmean08', 'tdmean09', 'tdmean10', 'tdmean11', 'tdmean12', 'tdmeanannual',

'tmax01', 'tmax02', 'tmax03', 'tmax04', 'tmax05', 'tmax06', 'tmax07', 'tmax08', 'tmax09', 'tmax10', 'tmax11', 'tmax12', 'tmaxannual',

'tmean01', 'tmean02', 'tmean03', 'tmean04', 'tmean05', 'tmean06', 'tmean07', 'tmean08', 'tmean09', 'tmean10', 'tmean11', 'tmean12', 'tmeanannual',

'tmin01', 'tmin02', 'tmin03', 'tmin04', 'tmin05', 'tmin06', 'tmin07', 'tmin08', 'tmin09', 'tmin10', 'tmin11', 'tmin12', 'tminannual',
                  'vpdmaxannual',

'vpdmin01', 'vpdmin02', 'vpdmin03', 'vpdmin04', 'vpdmin05', 'vpdmin06', 'vpdmin07', 'vpdmin08', 'vpdmin09', 'vpdmin10', 'vpdmin11', 'vpdmin12', 'vpdminannual']

knn_regress_model = KNeighborsRegressor(n_neighbors=3)


features = ['latstddeci','longstddec','elev']
for var in vars_to_impute:
    print(var)
    target = var
    knn_regress_model.fit(X = modeling_data.loc[modeling_data[target].notna(),features], 
                          y = modeling_data.loc[modeling_data[target].notna(),target])
    
    
    modeling_data.loc[modeling_data[target].isnull(), target] = knn_regress_model.predict(
                                modeling_data[features])[modeling_data[target].isnull()]

### Here are new climate variables to replace monthly variables:
    tdm_nov_to_apr = mean (tdmean11, tdmean12, tdmean01, tdmean02, tdmean03, tdmean04)
    tdm_may_to_oct = mean(tdmean05 ... tdmean10)
    ppt_may_to_sep = mean(ppt05 ... ppt09)
    ppt_oct_to_apr = mean(ppt10,ppt11,ppt12,ppt01,ppt02,ppt03,ppt04)
    tmax_apr_to_sep = mean(tmax04 ... tmax09)
    tmax_oct_to_mar = mean(tmax10,tmax11,tmax12,tmax01,tmax02,tmax03)
    tmin_apr_to_oct = mean(tmin04 ... tmin10)
    tmin_nov_to_mar = mean(tmin11, tmin12, tmin01, tmin02, tmin03)
    tmean_apr_to_oct = mean(tmean04 ... tmean10)
    tmean_nov_to_mar = mean(tmean11,tmean12,tmean01,tmean02,tmean03)
    vpdmin_jun_to_oct = mean(vpd06 ... vpd10)
    just drop vpdmax01 ... vpdmax12 (index is basically the same as vpdmaxannual

In [ ]:
modeling_data.drop(columns=['vpdmax01','vpdmax02','vpdmax03','vpdmax04',
                             'vpdmax05','vpdmax06','vpdmax07','vpdmax08',
                             'vpdmax09','vpdmax10','vpdmax11','vpdmax12'],inplace=True)

In [ ]:
def replace_var_with_average(var_list,model_data):
    avg = model_data[var_list].mean(axis=1)
    model_data.drop(columns=var_list,inplace=True)
    return(avg)

In [ ]:
modeling_data['tdm_nov_to_apr'] = replace_var_with_average(['tdmean11', 
                            'tdmean12', 'tdmean01', 'tdmean02', 'tdmean03', 'tdmean04'],modeling_data)
modeling_data['tdm_may_to_oct'] = replace_var_with_average(['tdmean05', 
                            'tdmean06', 'tdmean07', 'tdmean08', 'tdmean09', 'tdmean10'],modeling_data)

modeling_data['ppt_may_to_sep'] = replace_var_with_average(['ppt05','ppt06','ppt07','ppt08','ppt09'],modeling_data)
modeling_data['ppt_oct_to_apr'] = replace_var_with_average(['ppt10','ppt11','ppt12',
                                                             'ppt01','ppt02','ppt03','ppt04'],modeling_data)

modeling_data['tmax_apr_to_sep'] = replace_var_with_average(['tmax04','tmax05','tmax06','tmax07',
                                                              'tmax08','tmax09'],modeling_data)
modeling_data['tmax_oct_to_mar'] = replace_var_with_average(['tmax10','tmax11','tmax12','tmax01',
                                                              'tmax02','tmax03'], modeling_data)

modeling_data['tmin_apr_to_oct'] = replace_var_with_average(['tmin04','tmin05','tmin06','tmin07',
                                                              'tmin08','tmin09','tmin10'], modeling_data)
modeling_data['tmin_nov_to_mar'] = replace_var_with_average(['tmin11', 'tmin12', 'tmin01', 'tmin02',
                                                              'tmin03'], modeling_data)

modeling_data['tmean_apr_to_oct'] = replace_var_with_average(['tmean04','tmean05','tmean06','tmean07', 'tmean08','tmean09','tmean10'], modeling_data)
modeling_data['tmean_nov_to_mar'] = replace_var_with_average(['tmean11','tmean12','tmean01','tmean02',
                                                               'tmean03'], modeling_data)

modeling_data['vpdmin_jun_to_oct'] = replace_var_with_average(['vpdmin06','vpdmin07', 'vpdmin08',
                                                               'vpdmin09','vpdmin10'],modeling_data)

In [ ]:
modeling_data.drop_duplicates(inplace=True)

In [ ]:
modeling_data.shape, modeling_data.siteiid.nunique()

In [ ]:
ClusterGroups = pd.read_csv('Input Files/ClusterGroups.csv')
ClusterGroups.shape

In [ ]:
ClusterGroups.drop_duplicates(inplace=True)
ClusterGroups.shape

In [ ]:
modeling_data_with_groups = pd.merge(modeling_data,ClusterGroups,on='siteiid',how='left')
modeling_data_with_groups.shape

### Topographic Data

In [ ]:
topo_df_rest = pd.read_csv('Input Files/siteiid_lat_long_topo_20210319.csv')

VDPNED6_NEGNED6_df = topo_df_rest[['siteiid','VDPNED6','NEGNED6']].drop_duplicates()
VDPNED6_NEGNED6_df.shape

In [ ]:
site_siteobs_df = site_map_ids[['siteobsiid','siteiid']].drop_duplicates()

site_siteobs_df['siteobsiid'] = site_siteobs_df['siteobsiid'].astype(int)

site_siteobs_df.shape

In [ ]:
pedons_topo_df = pd.read_csv('Input Files/pedons_topo.csv')
pedons_topo_df = pd.merge(pedons_topo_df[['siteobsiid',
                        'CRVNED6','DEMNED6c','DVMNED6',
                        'GESUSG6_NA','MRNNED6','POSNED6',
                        'SLPNED6','TPINED6']],site_siteobs_df,on='siteobsiid',how='inner')
pedons_topo_df.shape

In [ ]:
pedons_topo_df = pedons_topo_df[['siteiid','CRVNED6','DEMNED6c','DVMNED6',
                        'GESUSG6_NA','MRNNED6','POSNED6',
                        'SLPNED6','TPINED6']].drop_duplicates()

pedons_topo_df['siteiid'] = pedons_topo_df['siteiid'].astype(int)

pedons_topo_df.shape

#### Fix GESUS variable with its classes

In [ ]:
surfacegeo = pd.read_csv('Input Files/surfacegeology_legend_final.csv')
surfacegeo.head()

In [ ]:
# pick out just the GESUS_variables and Value columns
surfacegeo_var = surfacegeo[['GESUS_variables', 'Value']]

In [ ]:
# replace original GESUS variables with its text version
pedons_topo_df_V2 = pd.merge(pedons_topo_df,
                      surfacegeo_var,
                      how='left',
                      left_on=['GESUSG6_NA'],
                      right_on=['Value'])
pedons_topo_df_V2

In [ ]:
# One Hot Encode GESUS_variables
GESUS_one_hot_encode = pedons_topo_df_V2[['siteiid']].join(pd.get_dummies(pedons_topo_df_V2['GESUS_variables'])).groupby('siteiid').max().reset_index()
GESUS_one_hot_encode

In [ ]:
# join one hot encoded variables back to dataset
pedons_topo_df_final = pd.merge(pedons_topo_df_V2,
                      GESUS_one_hot_encode,
                      how='left',
                      left_on=['siteiid'],
                      right_on=['siteiid'])
pedons_topo_df_final

In [ ]:
# Drop Value, GESUSG6_NA, GESUS_variables
pedons_topo_df_final = pedons_topo_df_final.drop(columns=['Value', 'GESUSG6_NA', 'GESUS_variables'])

#### merge data with topo data

In [ ]:
modeling_data_with_topography = pd.merge(modeling_data_with_groups,pedons_topo_df_final,on='siteiid',how='left')
modeling_data_with_topography.shape


In [ ]:
modeling_data_with_topography_cmp = pd.merge(modeling_data_with_topography,VDPNED6_NEGNED6_df,
                                              on='siteiid',how='left')
modeling_data_with_topography_cmp.shape

In [ ]:
# modeling_data_with_topography_cmp.head()
# modeling_data_with_topography_cmp[['VDPNED6','NEGNED6']].isna().mean()

#### Fill in NaN for surface geo one hot encoded variables

In [ ]:
surfacegeo_missing_var_list = [
'GESUS_alluvial_thick_sediments',
'GESUS_alluvial_thin_sediments',
'GESUS_coastal_zone_sendiments',
'GESUS_colluvial_alluvial_sediments',
'GESUS_colluvial_sediments_discontinuous',
'GESUS_colluvial_sediments_loess_residual_thin',
'GESUS_eolian_sediments_dunesand',
'GESUS_eolian_sediments_highplains',
'GESUS_eolian_sediments_loess',
'GESUS_glacial_till_sediments_clayey',
'GESUS_glacial_till_sediments_sandy',
'GESUS_glacial_till_sediments_silty',
'GESUS_glaciofluvial_icecontact_sediments',
'GESUS_organic_rich_muck',
'GESUS_other',
'GESUS_proglacial_sediments_coarse_grained',
'GESUS_proglacial_sediments_fine_grained',
'GESUS_residual_materials_alluvial_sediments',
'GESUS_residual_materials_bedrock',
'GESUS_residual_materials_carbonate_rocks',
'GESUS_residual_materials_fine_grained_sedimentary_rocks',
'GESUS_residual_materials_fine_igneous_metamorphic_rocks',
'GESUS_residual_materials_sedimentary_rocks',
'GESUS_water']
modeling_data_with_topography_cmp[surfacegeo_missing_var_list] = modeling_data_with_topography_cmp[
    surfacegeo_missing_var_list].apply(lambda x: x.fillna(0))

In [ ]:
modeling_data = modeling_data_with_topography_cmp

### Drop Index variable 
some index variables we drop after applying pca on model data and keep the pca value instead

In [ ]:
drop_pre_PCA_index_var = pd.read_excel('Input Files/VariablestoDropWhenUsingIndices.xlsx')

In [ ]:
drop_pre_PCA_vars = drop_pre_PCA_index_var['ExcludeVariables'].unique().tolist()

In [ ]:
shared_with_model = modeling_data.columns[modeling_data.columns.isin(drop_pre_PCA_vars)].tolist()

In [ ]:
list(set(drop_pre_PCA_vars) - set(shared_with_model))

In [ ]:
modeling_data.drop(columns= shared_with_model,inplace=True)
modeling_data.shape

In [ ]:
pca_indexed_vars = pd.read_csv('Input Files/ModelDataIndexData.csv')
pca_indexed_vars.shape

In [ ]:
pca_indexed_vars.drop_duplicates(inplace=True)
pca_indexed_vars.shape, pca_indexed_vars.siteiid.nunique()

In [ ]:
drop_climate_vars = pca_indexed_vars.columns[pca_indexed_vars.columns.str.contains('|'.join(['Temp',
                                                                           'VPD',
                                                                           'Precipitation',
                                                                           'Dewpoint']))].tolist()
pca_indexed_vars.drop(columns= drop_climate_vars,inplace=True)

In [ ]:
modeling_data.drop_duplicates(inplace=True)
modeling_data.shape, modeling_data.siteiid.nunique()

In [ ]:
modeling_data_with_pca_indexed = pd.merge(modeling_data,pca_indexed_vars,on='siteiid',how='left' )
modeling_data_with_pca_indexed.shape

### KNN Impute

In [ ]:
modeling_data = modeling_data_with_pca_indexed

In [ ]:
modeling_data.to_csv('Saved Datasets/modeling_data_with_pca_indexed.csv')

In [ ]:
# modeling_data = modeling_data_with_pca_indexed#.copy()


### these are the variables we replace with knn 
vars_to_impute = ['NDVI_5Pct', 'NDVI_IQR90', 'NDVI_95Pct', 'SATVI_5Pct', 'SATVI_IQR90', 'SATVI_95Pct',
                  'CRVNED6','DEMNED6c','DVMNED6', 'MRNNED6', 'POSNED6', 'SLPNED6',
                  'TPINED6', 'VDPNED6', 'NEGNED6']


knn_regress_model = KNeighborsRegressor(n_neighbors=3)


features = ['latstddeci','longstddec','elev']
for var in vars_to_impute:
    print(var)
    target = var
    knn_regress_model.fit(X = modeling_data.loc[modeling_data[target].notna(),features], 
                          y = modeling_data.loc[modeling_data[target].notna(),target])
    
    
    modeling_data.loc[modeling_data[target].isnull(), target] = knn_regress_model.predict(
                                modeling_data[features])[modeling_data[target].isnull()]

### Last Cleaning/Variable Drop

In [ ]:
# This is the modeling dataset that goes into the K-Means clustering algorithm
modeling_data.to_csv('Saved Datasets/modeling_data_afterKNN_2_17.csv',index=False)

In [ ]:
modeling_data.shape

In [ ]:
other_type_var = [x for x in list(modeling_data) if 'other' in x.lower()] + ['HorizonMasterOth0to10Index',
'HorizonMasterOth10to30Index',
'HorizonMasterOth40to70Index',
'HorizonMasterOth80to100Index',
'HorizonNameOth_30to120Index',
'HorizonTextureOTH_0to60Index',
'HorizonTextureOTH_60to120Index']

In [ ]:
other_type_var

In [ ]:
'pmgroupnam_OTHER',
'taxonname_OTHER',
'taxclname_OTHER',
'earthcov_1_Other grass/herbaceous cover',
'pmkind_OTHER',
'pmorigin_OTHER',
'Feature_other',
'PlantName_Other',
'HorizonMasterOth0to10Index',
'HorizonMasterOth10to30Index',
'HorizonMasterOth40to70Index',
'HorizonMasterOth80to100Index',
'HorizonNameOth_30to120Index',
'HorizonTextureOTH_0to60Index',
'HorizonTextureOTH_60to120Index',